In [3]:
import re
import gc
import os
import pickle
import random
# from preprocess import preprocess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split

In [4]:
# fix pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)


In [5]:
#load dataset digikala
df_ = pd.read_csv(r'./DK.csv', encoding='utf-8', engine='python', on_bad_lines='skip')

In [8]:
df_['len'] = df_['Comment'].astype(str).apply(lambda x: len(x.split()))
df_.head()

,Unnamed: 0.2,BadComment,Comment,Date,GoodComment,ItemURL,NegetiveFeedback,Person,PositiveFeedback,SubCatName,SubName,Unnamed: 0,Unnamed: 0.1,len
0,0,۰,<p>من کاملا با این محصول آشنا بودم و از خریدش مطمین بودم</p>,\r\n در تاریخ ۲ مهر ۱۳۹۹\r\n,۰,/product/dkp-1878144/ساعت-هوشمند-سامسونگ-مدل-galaxy-watch-active2-44mm,0,\r\n سمانه خلج مهرآبادي\r\n,1,ساعت هوشمند,کالای دیجیتال,NaN,NaN,12
1,1,۲,<p>به درد خاصی نمیخوره صرفا یکی دو روز باهاش سرگرمین بعد به عنوان یه ساعت معمولی ازش استفاده میکنین</p>,\r\n در تاریخ ۱ مهر ۱۳۹۹\r\n,۴,/product/dkp-1878144/ساعت-هوشمند-سامسونگ-مدل-galaxy-watch-active2-44mm,0,\r\n محجوبه زارعی\r\n,0,ساعت هوشمند,کالای دیجیتال,NaN,NaN,19
2,2,۰,<p>در یک کلام به نسبت هزینه ای که میکنید کالمل ترین ساعت بازار رو دریافت می‌کنید<br/>\r\r\nدسترسی کامل به نوتیفیکشن ها و توانایی به پاسخ دادن به پیام ها از طریق خود ساعت<br/>\r\r\nهمچنین برنامه کامل ورزشی تمام نیاز های شما رو برآورده میکنه</p>,\r\n در تاریخ ۳۱ شهریور ۱۳۹۹\r\n,۰,/product/dkp-1878144/ساعت-هوشمند-سامسونگ-مدل-galaxy-watch-active2-44mm,0,\r\n محمد رضا میرشکار سیاهکل\r\n,0,ساعت هوشمند,کالای دیجیتال,NaN,NaN,44
3,3,۰,<p>ساعت بسیار عالی و کاربردی هست،میخواستم برای دوستم هم بگیرم ولی به دلیل تفاوت قیمت زیاد از دیجی کالا نمیخرم،</p>,\r\n در تاریخ ۲۹ شهریور ۱۳۹۹\r\n,۰,/product/dkp-1878144/ساعت-هوشمند-سامسونگ-مدل-galaxy-watch-active2-44mm,0,\r\n کاربر دیجی‌کالا\r\n,0,ساعت هوشمند,کالای دیجیتال,NaN,NaN,20
4,4,۰,<p>دقیقا مطابق آنچه انتظار می رفت بود .<br/>\r\r\nخوب و کاربردی و مطابق استایل روز .</p>,\r\n در تاریخ ۲۹ شهریور ۱۳۹۹\r\n,۱,/product/dkp-1878144/ساعت-هوشمند-سامسونگ-مدل-galaxy-watch-active2-44mm,0,\r\n محمد میرفخرایی\r\n,1,ساعت هوشمند,کالای دیجیتال,NaN,NaN,16


In [9]:
df = df_[(df_['PositiveFeedback']==1)|(df_['NegetiveFeedback']==1)]


In [10]:
df['Label'] = 0
df.loc[df['NegetiveFeedback']==1,'Label'] = 1
df.head()[-20:]

C:\Users\Ehsan\AppData\Local\Temp\ipykernel_9488\3886734403.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Label'] = 0


,Unnamed: 0.2,BadComment,Comment,Date,GoodComment,ItemURL,NegetiveFeedback,Person,PositiveFeedback,SubCatName,SubName,Unnamed: 0,Unnamed: 0.1,len,Label
0,0,۰,<p>من کاملا با این محصول آشنا بودم و از خریدش مطمین بودم</p>,\r\n در تاریخ ۲ مهر ۱۳۹۹\r\n,۰,/product/dkp-1878144/ساعت-هوشمند-سامسونگ-مدل-galaxy-watch-active2-44mm,0,\r\n سمانه خلج مهرآبادي\r\n,1,ساعت هوشمند,کالای دیجیتال,NaN,NaN,12,0
4,4,۰,<p>دقیقا مطابق آنچه انتظار می رفت بود .<br/>\r\r\nخوب و کاربردی و مطابق استایل روز .</p>,\r\n در تاریخ ۲۹ شهریور ۱۳۹۹\r\n,۱,/product/dkp-1878144/ساعت-هوشمند-سامسونگ-مدل-galaxy-watch-active2-44mm,0,\r\n محمد میرفخرایی\r\n,1,ساعت هوشمند,کالای دیجیتال,NaN,NaN,16,0
5,5,۰,<p>خیییییییییییییییلی خوبه من تقریبا 8 ماهه دارمش خیلی خوبه</p>,\r\n در تاریخ ۲۸ شهریور ۱۳۹۹\r\n,۰,/product/dkp-1878144/ساعت-هوشمند-سامسونگ-مدل-galaxy-watch-active2-44mm,0,\r\n مریم کیال\r\n,1,ساعت هوشمند,کالای دیجیتال,NaN,NaN,9,0
6,6,۱,<p>بسیار عالی خدایی کار باهاش بسیار راحته . فقط قیمتش رو کمتر شه بهتره یکی دیگه باید بگیرم</p>,\r\n در تاریخ ۲۷ شهریور ۱۳۹۹\r\n,۲,/product/dkp-1878144/ساعت-هوشمند-سامسونگ-مدل-galaxy-watch-active2-44mm,0,\r\n وحید عارفی کیان\r\n,1,ساعت هوشمند,کالای دیجیتال,NaN,NaN,18,0
8,8,۲,<p>بسیار ساده و شیک و سبک است و کار کردن با آن آسان است</p>,\r\n در تاریخ ۲۶ شهریور ۱۳۹۹\r\n,۳,/product/dkp-1878144/ساعت-هوشمند-سامسونگ-مدل-galaxy-watch-active2-44mm,0,\r\n علیا شریعتی\r\n,1,ساعت هوشمند,کالای دیجیتال,NaN,NaN,14,0


In [11]:
# انتخاب ۷۰۰ داده که Label برابر با ۰ دارند
zero_label_samples = df[df['Label'] == 0].head(308112)

# انتخاب بقیه داده‌ها (داده‌هایی که Label برابر با ۱ دارند)
one_label_samples = df[df['Label'] == 1]

# ترکیب داده‌ها
df_filtered = pd.concat([zero_label_samples, one_label_samples])

# اگر می‌خواهید داده‌ها را به صورت تصادفی ترکیب کنید:
df_filtered = df_filtered.sample(frac=1).reset_index(drop=True)

print(len(df_filtered))  # تعداد کل داده‌ها

616224


In [12]:
X = df_filtered['Comment'].values
Y = df_filtered['Label'].values

In [13]:
count_0 = (df_filtered['Label'] == 0).sum()
count_1 = (df_filtered['Label'] == 1).sum()

print("تعداد 0 ها:", count_0)
print("تعداد 1 ها:", count_1)
# df =df_filtered

تعداد 0 ها: 308112
تعداد 1 ها: 308112


In [14]:
print(len(df_filtered['Label'].values==1))

616224


In [40]:
pip install hazm


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [322]:
from hazm import *
import pandas as pd

# ایجاد شیء‌های لازم
normalizer = Normalizer()
stemmer = Stemmer()
lemmatizer = Lemmatizer()
tokenizer = WordTokenizer()  # اضافه کردن توکنایزر Hazm

# تعریف تابع پردازش متن
def process_text(text):
    # نرمال‌سازی متن
    normalized_text = normalizer.normalize(text)
    
    # توکن‌سازی کلمات با استفاده از توکنایزر Hazm
    words = tokenizer.tokenize(normalized_text)
    
    # ریشه‌یابی و لم‌سازی کلمات
    processed_words = [lemmatizer.lemmatize(stemmer.stem(word)) for word in words]
    
    # تبدیل کلمات پردازش‌شده به یک رشته متنی
    processed_text = ' '.join(processed_words)
    
    return processed_text

# تعداد ردیف‌های مورد نظر
num_rows = 30000

# انتخاب ردیف‌های اولیه از DataFrame
batch_df = df.iloc[:num_rows].copy()

# پردازش ستون Comment و ذخیره نتیجه در ستون جدید
batch_df['Processed_Comment'] = batch_df['Comment'].apply(process_text)

# نمایش ۵ ردیف اول برای بررسی
print(batch_df.head())

   Unnamed: 0.2 BadComment  \
0             0          ۰   
4             4          ۰   
5             5          ۰   
6             6          ۱   
8             8          ۲   

                                                                                   Comment  \
0                                    من کاملا با این محصول آشنا بودم و از خریدش مطمین بودم   
4                  دقیقا مطابق آنچه انتظار می رفت بود . خوب و کاربردی و مطابق استایل روز .   
5                                   خیییییییییییییییلی خوبه من تقریبا ماهه دارمش خیلی خوبه   
6  بسیار عالی خدایی کار باهاش بسیار راحته . فقط قیمتش رو کمتر شه بهتره یکی دیگه باید بگیرم   
8                                     بسیار ساده و شیک و سبک است و کار کردن با آن آسان است   

                                                                                          Date  \
0      \r\n                                در تاریخ ۲ مهر ۱۳۹۹\r\n                               
4  \r\n                                در تاریخ ۲۹ شهریور 

In [324]:
# batch_df['Processed_Comm؟ent']

In [326]:

# X_processed, df =df['Comment']

ValueError: too many values to unpack (expected 2)

In [328]:
 batch_df.keys()

Index(['Unnamed: 0.2', 'BadComment', 'Comment', 'Date', 'GoodComment',
       'ItemURL', 'NegetiveFeedback', 'Person', 'PositiveFeedback',
       'SubCatName', 'SubName', 'Unnamed: 0', 'Unnamed: 0.1', 'len', 'Label',
       'Processed_Comment'],
      dtype='object')

In [331]:
X_processe

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [333]:
X_processed = batch_df['Processed_Comment']

In [335]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# تبدیل متن به بردارهای TF-IDF
vectorizer = TfidfVectorizer(max_features=2000)  # محدودیت تعداد ویژگی‌ها به 5000
X_vectorized = vectorizer.fit_transform(X_processed).toarray()


In [337]:
Y =batch_df['Label']
len(Y)

30000

In [339]:
X_vectorized = X_vectorized.astype('float32')
# تقسیم داده‌ها به آموزش و تست (70% آموزش، 30% تست)
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, Y, test_size=0.3, random_state=1)

# آموزش مدل Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

# پیش‌بینی روی داده‌های تست
y_pred_nb = nb_model.predict(X_test)

# ارزیابی مدل
accuracy_nb = accuracy_score(y_test, y_pred_nb)
print("دقت مدل Naive Bayes:", accuracy_nb)
print(classification_report(y_test, y_pred_nb))

دقت مدل Naive Bayes: 0.89
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      7603
           1       0.89      0.33      0.48      1397

    accuracy                           0.89      9000
   macro avg       0.89      0.66      0.71      9000
weighted avg       0.89      0.89      0.87      9000



In [341]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train, y_train)

# Predict Output
predicted = model.predict(X_test)

print("Actual Value:", y_test[300])
print("Predicted Value:", predicted[300])

Actual Value: 0
Predicted Value: 1


In [149]:
accuracy_nb = accuracy_score(y_test, predicted)


In [151]:
accuracy_nb

0.5833333333333334

In [148]:
import matplotlib.pyplot as plt
plt.subplot(accuracy_nb)

ValueError: Single argument to subplot must be a three-digit integer, not 0.8408864703464093

<Figure size 640x480 with 0 Axes>